# RQ3 Analysis: Poverty, Stunting, and Diet

This notebook performs the regression analysis from `RQ3.do`.

In [ ]:
!pip install survey openpyxl statsmodels

In [ ]:
import pandas as pd
from survey import Survey
import statsmodels.formula.api as smf
import numpy as np
import os
from openpyxl import Workbook

## Part 1: Poverty Analysis

### Load and Prepare Data

In [ ]:
hr_df = pd.read_csv(os.path.join('..', 'data', 'cleaned', 'hrdata_clean.csv'))

# Create include variable
hr_df['include'] = 0
hr_df.loc[hr_df['region'].isin([25, 51, 52, 53]), 'include'] = 1

# Recode region
region_map = {
    52: "Androy",
    53: "Anosy",
    51: "AtsimoAndrefana",
    25: "AtismoAtsinanana"
}
hr_df['newregion'] = hr_df['region'].map(region_map)

# Filter for subpopulation
hr_subpop_df = hr_df[hr_df['include'] == 1].copy()

# Create dependency ratio
hr_subpop_df['num_mbrs_dpndt'] = hr_subpop_df['num_mbrs_14'] + hr_subpop_df['num_mbrs_65']

### Survey Design

In [ ]:
hr_survey_design = Survey(hr_subpop_df, weights='wt', strata='hv022', psu='hv021')

### Generic Regression Function

In [ ]:
def run_regression_analysis(data, survey_design, outcome_var, independent_vars, regions, excel_writer, sheet_name):
    results = []
    for var in independent_vars:
        row = {'Variable': var}
        for region_code, region_name in regions.items():
            formula = f'{var} ~ {outcome_var}'
            try:
                region_data = data[data['newregion'] == region_name]
                model = smf.wls(formula, data=region_data, weights=region_data['wt']).fit()
                row[f'{region_name}_coef'] = model.params[outcome_var]
                row[f'{region_name}_se'] = model.bse[outcome_var]
                row[f'{region_name}_p'] = model.pvalues[outcome_var]
            except Exception as e:
                print(f"Could not run regression for {var} in {region_name}: {e}")
        results.append(row)
    
    results_df = pd.DataFrame(results)
    results_df.to_excel(excel_writer, sheet_name=sheet_name, index=False)

### Run Poverty Analysis and Export

In [ ]:
output_excel_path_pov = os.path.join('..', 'data', 'cleaned', 'RQ3_Poverty_Analysis.xlsx')
writer_pov = pd.ExcelWriter(output_excel_path_pov, engine='openpyxl')

regions = {1: 'Androy', 2: 'Anosy', 3: 'AtsimoAndrefana', 4: 'AtismoAtsinanana'}

# Household Demographics
hh_demo_vars = ['hh_head_female', 'hh_head_age', 'hhsize', 'num_child_under5', 'num_mbrs_dpndt', 'dependency_ratio']
run_regression_analysis(hr_subpop_df, hr_survey_design, 'poverty20', hh_demo_vars, regions, writer_pov, 'HH_Demographics')

# Socio-economic Characteristics
socio_econ_vars = ['educ_no_education', 'educ_incompl_primary', 'educ_primary', 'educ_incompl_sec1', 'educ_sec1', 'educ_incompl_sec2', 'educ_sec2', 'educ_higher', 'educ_dk', 'water_improve', 'toilet_improved', 'toilet_unimproved', 'toilet_open', 'roof_thatchleaf', 'roof_palmbamb', 'roof_metal', 'roof_other', 'floor_earthsand', 'floor_woodplanks', 'floor_palmbamb', 'floor_mats', 'floor_vinylasph', 'floor_cement', 'floor_other', 'electricity', 'cookfuel_charcoal', 'cookfuel_wood', 'cookfuel_strawshrub', 'cookfuel_other', 'num_rooms_sleep', 'radio', 'television', 'mobile_phone', 'bicycle', 'urban']
run_regression_analysis(hr_subpop_df, hr_survey_design, 'poverty20', socio_econ_vars, regions, writer_pov, 'SocioEconomic')

# Assets
asset_vars = ['own_agricland', 'animal_cart', 'hectares_agricland', 'own_livestock', 'own_cows', 'own_horses', 'own_goats', 'own_sheep', 'own_poultry', 'own_zebus', 'own_ducks', 'own_pigs', 'numown_poultry', 'numown_zebus']
run_regression_analysis(hr_subpop_df, hr_survey_design, 'poverty20', asset_vars, regions, writer_pov, 'Assets')

writer_pov.close()

## Part 2: Child Stunting Analysis

### Load and Prepare Data

In [ ]:
stunt_df = pd.read_csv(os.path.join('..', 'data', 'cleaned', 'child_stunting_analysis.csv'))

# Create include variable
stunt_df['include'] = 0
stunt_df.loc[stunt_df['region'].isin([25, 51, 52, 53]), 'include'] = 1

# Recode region
stunt_df['newregion'] = stunt_df['region'].map(region_map)

# Filter for subpopulation
stunt_subpop_df = stunt_df[stunt_df['include'] == 1].copy()

# Create dependency ratio
stunt_subpop_df['num_mbrs_dpndt'] = stunt_subpop_df['num_mbrs_14'] + stunt_subpop_df['num_mbrs_65']

### Survey Design

In [ ]:
stunt_survey_design = Survey(stunt_subpop_df, weights='wt', strata='hv022', psu='hv021')

### Run Stunting Analysis and Export

In [ ]:
output_excel_path_stunt = os.path.join('..', 'data', 'cleaned', 'RQ3_Stunting_Analysis.xlsx')
writer_stunt = pd.ExcelWriter(output_excel_path_stunt, engine='openpyxl')

# Household Demographics
run_regression_analysis(stunt_subpop_df, stunt_survey_design, 'nt_ch_stunt', hh_demo_vars, regions, writer_stunt, 'HH_Demographics')

# Socio-economic Characteristics
run_regression_analysis(stunt_subpop_df, stunt_survey_design, 'nt_ch_stunt', socio_econ_vars, regions, writer_stunt, 'SocioEconomic')

# Assets
run_regression_analysis(stunt_subpop_df, stunt_survey_design, 'nt_ch_stunt', asset_vars, regions, writer_stunt, 'Assets')

writer_stunt.close()

## Part 3: Child Diet Analysis

### Load and Prepare Data

In [ ]:
diet_df = pd.read_stata(os.path.join('..', 'data', 'cleaned', 'child_diet_analysis.dta'))

# Create include variable
diet_df['include'] = 0
diet_df.loc[diet_df['region'].isin([25, 51, 52, 53]), 'include'] = 1

# Recode region
diet_df['newregion'] = diet_df['region'].map(region_map)

# Filter for subpopulation
diet_subpop_df = diet_df[diet_df['include'] == 1].copy()

# Create dependency ratio
diet_subpop_df['num_mbrs_dpndt'] = diet_subpop_df['num_mbrs_14'] + diet_subpop_df['num_mbrs_65']

# Create mother occupation dummies
recode_map = {0: 'Not working', 4: 'Agriculture', 5: 'Agriculture', 1: 'Other', 2: 'Other', 3: 'Other', 6: 'Other', 7: 'Other', 8: 'Other', 10: 'Other', 96: 'Other'}
diet_subpop_df['mother_occup'] = diet_subpop_df['v717'].map(recode_map)
occup_dummies = pd.get_dummies(diet_subpop_df['mother_occup'], prefix='occup')
diet_subpop_df = pd.concat([diet_subpop_df, occup_dummies], axis=1)

### Survey Design

In [ ]:
diet_survey_design = Survey(diet_subpop_df, weights='wt', strata='hv022', psu='hv021')

### Run Diet Analysis and Export

In [ ]:
output_excel_path_diet = os.path.join('..', 'data', 'cleaned', 'RQ3_Diet_Analysis.xlsx')
writer_diet = pd.ExcelWriter(output_excel_path_diet, engine='openpyxl')

# Household Demographics
run_regression_analysis(diet_subpop_df, diet_survey_design, 'nt_mad', hh_demo_vars, regions, writer_diet, 'HH_Demographics')

# Socio-economic Characteristics
diet_socio_econ_vars = socio_econ_vars + ['occup_Not working', 'occup_Agriculture', 'occup_Other', 'lit_cat', 'ch_diar', 'ch_ari']
diet_subpop_df['lit_cat'] = pd.cut(diet_subpop_df['v155'], bins=[-1, 0, 2], labels=['None', 'Read part or whole sentence'], right=True)
run_regression_analysis(diet_subpop_df, diet_survey_design, 'nt_mad', diet_socio_econ_vars, regions, writer_diet, 'SocioEconomic')

# Assets
run_regression_analysis(diet_subpop_df, diet_survey_design, 'nt_mad', asset_vars, regions, writer_diet, 'Assets')

writer_diet.close()